# COMP0005 - GROUP COURSEWORK
# Experimental Evaluation of Search Data Structures and Algorithms

The cell below defines **AbstractSearchInterface**, an interface to support basic insert/search operations; you will need to implement this three times, to realise your three search data structures of choice among: (1) *2-3 Tree*, (2) *AVL Tree*, (3) *LLRB BST*; (4) *B-Tree*; and (5) *Scapegoat Tree*. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [11]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractSearchInterface(ABC):
    '''
    Abstract class to support search/insert operations (plus underlying data structure)
    
    '''
        
    @abstractmethod
    def insertElement(self, element):     
        '''
        Insert an element in a search tree
            Parameters:
                    element: string to be inserted in the search tree (string)

            Returns:
                    "True" after successful insertion, "False" if element is already present (bool)
        '''
        
        pass 
    

    @abstractmethod
    def searchElement(self, element):
        '''
        Search for an element in a search tree
            Parameters:
                    element: string to be searched in the search tree (string)

            Returns:
                    "True" if element is found, "False" otherwise (bool)
        '''

        pass

Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [12]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE



Use the cell below to implement the requested API by means of **2-3 Tree** (if among your chosen data structure).

In [13]:
class TwoThreeTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found    

Use the cell below to implement the requested API by means of **AVL Tree** (if among your chosen data structure).

In [14]:
class AVLTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found  

Use the cell below to implement the requested API by means of **LLRB BST** (if among your chosen data structure).

In [15]:
class LLRBBST(AbstractSearchInterface):

    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, value, colour=True):
            self.value = value
            self.left = None
            self.right = None
            self.colour = colour 
            # where True = red and False = black

    def rotate_left(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = True
        return x

    def rotate_right(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = True
        return x

    def colour_flip(self, node):
        node.colour = True
        node.left.colour = False
        node.right.colour = False
        return node
    
    def _get(self, node, value):
        if node is None:
            return None
        
        # classic BST search
        if value < node.value:
            return self._get(node.left, value)
        elif value > node.value:
            return self._get(node.right, value)
        else:
            return node.value  

    def get(self, value):
        return self._get(self.root, value)

    def _set(self, node, value):
        if node is None:
            return self.Node(value, True)  
        
        # classic BST insert 
        if value < node.value:
            node.left = self._set(node.left, value)
        elif value > node.value:
            node.right = self._set(node.right, value)
        else:
            node.value = value

        # now balance the tree
        if node.right is not None and node.right.colour == True and (node.left is None or node.left.colour == False):
            node = self.rotate_left(node)
        if node.left is not None and node.left.colour == True and node.left.left is not None and node.left.left.colour == True:
            node = self.rotate_right(node)
        if node.left is not None and node.left.colour == True and node.right is not None and node.right.colour == True:
            node = self.colour_flip(node)

        return node
        
    def set(self, value):
        self.root = self._set(self.root, value)
        self.root.colour = False  # root is always black
 
    def insertElement(self, element):

        # element already exists
        if self.get(element) is not None:
            return False  

        # insert the element
        self.set(element)
        return True

    def searchElement(self, element):

        return self.get(element) is not None
    



tree = LLRBBST()
tree.insertElement("hello")
tree.insertElement("why")
print(tree.searchElement("hello"))
print(tree.searchElement("no"))

Use the cell below to implement the requested API by means of **B-Tree** (if among your chosen data structure).

In [16]:
class BTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found

Use the cell below to implement the requested API by means of **Scapegoat Tree** (if among your chosen data structure).

In [17]:
class ScapegoatTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found 

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (be mindful of code readability and reusability).

In [18]:
import string
import random

class TestDataGenerator():
    '''
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
    
    #ADD YOUR CODE HERE
    
    def __init__():
        pass
    

Use the cell below to implement the requested **experimental framework** (be mindful of code readability and reusability).

In [19]:
import timeit
import matplotlib

class ExperimentalFramework():
    '''
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
            
    #ADD YOUR CODE HERE
    
    def __init__():
        pass
    

Use the cell below to illustrate the python code you used to **fully evaluate** your three chosen search data structures and algortihms. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each data structure using such data, collect information about their execution time, plot results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [20]:
# ADD YOUR TEST CODE HERE 



